In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
#from supervised.automl import AutoML

ModuleNotFoundError: No module named 'supervised'